In [1]:
# Human specification
import inspect

environment_name = "TwoAgentsVault"
specification = inspect.cleandoc("""\
                Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. 
                Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, and only interact with one block at a time. 
                Additionally, one agent can only interact with vowel blocks, and the other can only interact with consonant blocks. 
                Initially there are blocks A, B, C, O. A is on the table. B is on top of A, and C and O are on the table. 
                The goal is to have A on the table, B on the table, C on top of O, and O on top of B.
                """)  # This prompt is ignored if the environment_name already corresponds to an existing environment.
format = "json"

In [2]:
# Generate the json representation of the environment
from pathlib import Path

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT
from src.llm_plan.config import ENVIRONMENTS_JSON_PATH

model_json = ChatGPT("gpt-5-mini")  # This is one of the many models you can use now
planner = Planner()

plan_path = Path(f"{environment_name}.{format}")
full_path = ENVIRONMENTS_JSON_PATH / plan_path

# Skip if the plan already exists
if not full_path.exists():
    planner.generate_representation(model_json, specification, environment_name, format=format)
else:
    print(f"{full_path} already exists. Skipping generation.")
    print("[Warning]: The prompt `specification` will be ignored!")

environments/static/TwoAgentsVault.json already exists. Skipping generation.
[Warning]: The prompt `specification` will be ignored!


In [3]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
print("Plan:\n", env.plan)

Plan:
 [['key_holder.pddl', 'small_robot.pddl'], ['orchestrator.pddl']]


In [4]:
# Start generating the first domain and problem
import re
import subprocess
from pathlib import Path

from src.llm_plan.hypervisor import Hypervisor
from src.llm_plan.config import VALIDATOR_BIN, SOLVER_POPF2_BINARY
from src.llm_plan.parser import PDDLParser
from src.llm_plan.utils import run_pddl_popf2_and_Val

model_first_plan = ChatGPT("gpt-5-mini")  # powerful model for the first plan
model_plan = ChatGPT("gpt-4o")  # switch to a less powerful model for planning

pddl_parser = PDDLParser()

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

# Fist plan to collect domain and problem
# Check if problem_0 and domain_0 already exist
if (BASE_FOLDER / f"problem_0.pddl").exists() and (BASE_FOLDER / f"domain_0.pddl").exists():
    print(f"A plan for {env.name} already exist. Loading domain and problem.")
    with open(BASE_FOLDER / f"domain_0.pddl", "r") as f:
        domain = f.read()
    with open(BASE_FOLDER / f"problem_0.pddl", "r") as f:
        problem = f.read()

else:
    print("Generating the first plan.")
    responses = planner.plan(model_first_plan, env)
    final_plan = responses["pddl_orchestrator"]
    domain, problem = pddl_parser.parse(final_plan, from_file=False)
    
    
# Generate the first plan
result = run_pddl_popf2_and_Val(
    BASE_FOLDER / f"domain_0.pddl",
    BASE_FOLDER / f"problem_0.pddl",
    BASE_FOLDER / f"sas_plan_0",
)

Generating the first plan.
STDOUT:
 
STDERR:
 
Return Code: -11 <class 'int'>


In [7]:
# Refine the plan

# Args for the hypervisor
budget = 5
base_agent = "AgentDeepThinkPDDL"
prompt_args_hypervisor = {
    "human_specification": specification,
    "plan": "No plan yet.",
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": result["pddl_plan"],
    "syntax_errors": result["syntax_errors"],
    "pddl_logs": result["pddl_logs"],
    "history": [],
}

for j in range(1, budget+1):
    print(prompt_args_hypervisor)
    hypervisor = Hypervisor(prompt_args_hypervisor)
    response = hypervisor.run(model_plan)
    
    # Dynamically instantiate the agent class
    match = re.search(r"<class>(.*?)</class>", response, re.DOTALL)
    if match:
        agent_name = match.group(1).strip()
    else:
        print(f"[Warning] No agent class found in the response ({agent_name}). Using default {base_agent}.")
        agent_name = base_agent
        
    print("Agent: ", agent_name)
    print("Generating the plan with the agent.")
    agent_class = hypervisor.agents[agent_name]
    
    required_args = {}
    for arg in agent_class.required_args.keys():
        agent_class.required_args[arg] = prompt_args_hypervisor[arg]
        
    new_agent = agent_class(model_plan, agent_class.required_args)
    response = new_agent.run()
    
    # The Hypervisor decides the plan is good
    if agent_name == "NoOpAgent":
        break
    
    # Get domain and plan
    domain, problem = pddl_parser.parse(response, from_file=False)
    prompt_args_hypervisor["pddl_domain"] = domain
    prompt_args_hypervisor["pddl_problem"] = problem
    
    # Run the pddl planner
    with open(BASE_FOLDER / f"problem_{j}.pddl", "w") as f:
        f.write(str(problem))

    with open(BASE_FOLDER / f"domain_{j}.pddl", "w") as f:
        f.write(str(domain))

    result = run_pddl_popf2_and_Val(
        BASE_FOLDER / f"domain_{j}.pddl",
        BASE_FOLDER / f"problem_{j}.pddl",
        BASE_FOLDER / f"sas_plan_{j}",
    )

    # Update the hypervisor args
    for k, v in result.items():
        prompt_args_hypervisor[k] = v

    # Update the history
    prompt_args_hypervisor["history"].append(hypervisor.history)

{'human_specification': 'Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. \nEach block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, and only interact with one block at a time. \nAdditionally, one agent can only interact with vowel blocks, and the other can only interact with consonant blocks. \nInitially there are blocks A, B, C, O. A is on the table. B is on top of A, and C and O are on the table. \nThe goal is to have A on the table, B on the table, C on top of O, and O on top of B.', 'plan': 'No plan yet.', 'specification': {'name': 'TwoAgentsVault', 'author': 'Human', 'agents': {'number': 3, 'names': ['key_holder', 'small_robot', 'orchestrator'], 'key_holder': {'private_information': ['I have a key', 'I am a big robot'], 'goal': 'Open the vault and grab the object inside.'}, 'small_robot': {'private_information': ['I am a small robot'], 'goal': 'Open the vault an

In [ ]:
from src.llm_plan.agent import AgentNaturalLanguage

# Produce the natural language plan
if Path(BASE_FOLDER / f"sas_plan_{j}").exists():
    with open(BASE_FOLDER / f"domain_{j}.pddl", "r") as f:
        domain = f.read()

    with open(BASE_FOLDER / f"problem_{j}.pddl", "r") as f:
        problem = f.read()

    with open(BASE_FOLDER / f"sas_plan_{j}", "r") as f:
        plan = f.read()

    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "pddl_plan": plan,
    }

    hypervisor_to_nl = AgentNaturalLanguage(
        llm=model_plan, prompt_args=prompt_args
    )

    natural_plan = hypervisor_to_nl.run()

    with open(BASE_FOLDER / "final_natural_plan.txt", "w") as f:
        f.write(natural_plan)